In [ ]:
# Función de coste binaria con valores {-1, 1}
# g(z) = "\
#     +11 * z_0 - 17.5 * z_1 - 28 * z_2 - 17 * z_3 + 11.5 * z_4 \
#     +13.5 * (-z_0*z_2 + z_1*z_2 + z_2*z_3 - z_2*z_4 + z_0*z_1 - z_0*z_3 - z_1*z_4) \
#     +(6.75*z_3**2 + 6.75*z_4**2 + 13.5*(z_0**2 + z_1**2 + z_2**28)) \
#     +26.5"

# Grafo
edges = {(0, 1): 5, (0, 2): 8, (1, 2): 2, (1, 3): 7, (2, 3): 4}
num_nodes = 4

# Hp = 11*Z0 - 17.5*Z1 - 28*Z2 - 17*Z3 + 11.5*Z4 +
#      13.5*(-Z0*Z2 + Z1*Z2 + Z2*Z3 - Z2*Z4 + Z0*Z1 - Z0*Z3 - Z1*Z4)

# Coeficientes lineales de Hp (11 * Z0 - 17.5 * Z1...)
linear_coefs = [11, -17.5, -28, -17, 11.5]

# Coeficientes cuadráticos Ej: {(2, 3): 13.5} -> 13.5 * Z2 * Z3
quadra_coefs = {(0, 2): -13.5, (1, 2): 13.5, (2, 3): 13.5, (2, 4): -13.5,
                (0, 1): 13.5, (0, 3): -13.5, (1, 4): -13.5}

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Imprime un grafo dirigido pesado
# Entrada
#     graph_nodes: Lista de nodos del grafo
#     graph_edges: Diccionario {vértice: peso} donde vértice es (n1, n2)
#     layout:      Para especificar la posición de los nodos
def print_graph(graph_nodes, graph_edges, layout=None):
    G = nx.DiGraph()

    # Nodes
    G.add_nodes_from(graph_nodes)

    # Edges
    for pair, weight in edges.items():
        G.add_edge(pair[0], pair[1], weight=weight)

    # Print graph
    if layout is None:
        layout = nx.spring_layout(G)

    plt.figure(3, figsize=(6, 2))
    nx.draw(G, layout, with_labels=True, node_size=600)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)
    # plt.savefig("../../latex/resultados/img/primer_grafo.png")

# edges = {(0, 1): "5(q_0)", (0, 2): "8(q_1)", (1, 2): "2(q_2)", (1, 3): "7(q_3)", (2, 3): "4(q_4)"}
print_graph(range(num_nodes), edges, layout={0: [-1, 0], 1: [0, 1],
                                             2: [0, -1], 3: [1, 0]})

In [ ]:
# Función de coste binaria con valores {0, 1}
cost_function = "\
5*x_01 + 8*x_02 + 2*x_12 + 7*x_13 + 4*x_23 + \
27*(x_01 + x_02 - 1)**2 + \
27*(x_01 - x_12 - x_13)**2 + \
27*(x_02 + x_12 - x_23)**2"

# Evalúa una solución concreta según la función de coste para la versión QUBO
# Entrada:
#     - bits: Cadena con los valores de los qubits medidos
#             Orden: q_4 q_3 q_2 q_1 q_0
def eval_cost_function(bits):
    assert len(edges) == len(bits), "Error in cost_function: Length of bits"

    inv_bits = bits[::-1]  # Los qubits están en orden inverso

    param_dict = {}
    # Asignación entre x_ij -> q_n
    # Establecido por el orden de edges. Ej: x_12 -> q_2 (esto es, qubit nº2)
    for n, (i, j) in enumerate(edges):
        param_dict[f"x_{i}{j}"] = int(inv_bits[n])

    return eval(cost_function, param_dict)

In [ ]:
from qiskit import QuantumCircuit
def generate_qaoa_circuit(theta):
    assert len(theta) % 2 == 0, "Error in parameters (Beta, Gamma)"

    nqubits = len(edges)  # Tantos qubits como aristas tenga el grafo
    circuit = QuantumCircuit(nqubits)

    layers = int(len(theta) / 2)
    beta = theta[:layers]
    gamma = theta[layers:]

    # |v0>
    for i in range(nqubits):
        circuit.h(i)

    for p in range(layers):
        circuit.barrier()

        # Hp
        for q_idx, coef in enumerate(linear_coefs):
            circuit.rz(coef, q_idx)
        for q_idxs, coef in quadra_coefs.items():
            circuit.rzz(coef * gamma[p], q_idxs[0], q_idxs[1])

        circuit.barrier()

        # Hm
        for q_idx in range(nqubits):
            circuit.rx(beta[p] * 2, q_idx)

    circuit.measure_all()
    return circuit

In [ ]:
def compute_expectation(counts):
    media = 0
    len_count = 0
    for bits, count in counts.items():
        cost = eval_cost_function(bits)
        media += cost * count
        len_count += count

    return media/len_count

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
# Save an IBM Quantum account on disk
#QiskitRuntimeService.save_account(
#    channel="ibm_quantum",
#    token="",
#    overwrite=True
#)

In [ ]:
# Print all available computers and the number of current jobs in each one
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")
backends = [(b.name, b.status().pending_jobs)
            for b in service.backends(simulator=False, operational=True)]
backends = sorted(backends, key=lambda x: x[1])
for back in backends:
    print(f"{back[0]} {back[1]}")

In [ ]:
# Ejecución
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session, Options
from scipy.optimize import minimize

service = QiskitRuntimeService(channel="ibm_quantum")
options = Options()
options.execution.shots = 1024

def execute_circuit(theta):
    qc = generate_qaoa_circuit(theta)
    job = Sampler(options=options).run(qc)
    result = job.result()
    counts = result.quasi_dists[0].binary_probabilities(len(edges))
    return compute_expectation(counts)

with Session(backend=service.backend("ibmq_lima")):
    num_layers = 1
    theta_res = minimize(execute_circuit, [0.5, 0.5] * num_layers, method = "COBYLA")
    print(theta_res)

In [ ]:
from qiskit import Aer
from qiskit.visualization import plot_histogram

backend = Aer.get_backend('aer_simulator')
shots = 1024

qc = generate_qaoa_circuit([0.6869, 0.4728])
counts = backend.run(qc, shots=shots).result().get_counts()
plot_histogram(counts, figsize=(10, 7),
               filename="/home/vian/0_uam/1_TFG/latex/resultados/img/primer_provider_lima.png")
#plot_histogram(counts, figsize=(10, 7))